# Problem 6: Comprehensive WAF Implementation and Testing

### Intro


The goal of this problem is to implement and evaluate a defense-in-depth approach using a WAF. By leveraging ModSecurity with the OWASP Core Rule Set (CRS), this exercise focuses on securing web applications against a variety of attacks. The key tasks involve configuring the WAF, creating custom rules, testing against multiple security levels, and tuning the system to minimize false positives while maximizing security effectiveness. This process aims to protect against vulnerabilities such as SQL injection, XSS, command injection, and other malicious threats, enhancing the overall security of web applications.

### Complete ModSecurity configuration

![Hash passwords](imag/6_1.jpeg)

![Hash passwords](imag/6_2.jpeg)

**ModSecurity Logging and Monitoring Plan**

To enable ModSecurity logging, three key directives need to be configured in the `modsecurity.conf` file. First, **SecAuditEngine On** activates the audit engine, allowing all HTTP transactions processed by the firewall to be logged. The **SecAuditLog** directive specifies the log file location, in this case, `/var/log/modsec_audit.log`. Lastly, **SecAuditLogParts ABIFHZ** defines which parts of the HTTP transaction will be logged, such as request headers, response status, and other essential components for security analysis. This configuration ensures comprehensive monitoring of all transactions, enabling administrators to identify malicious activity, attack patterns, and potential vulnerabilities in the system.


![Hash passwords](imag/6_3.jpeg)

![Hash passwords](imag/6_4.jpeg)

## Set Up Real-Time Monitoring

To enhance real-time monitoring, the `fail2ban` service is installed and configured to detect and block malicious activity based on ModSecurity logs. First, Fail2Ban is installed using the command `sudo apt-get install fail2ban`. Then, the ModSecurity filter is configured by editing the `/etc/fail2ban/filter.d/modsec.conf` file, where a regular expression is added to detect denied requests in the ModSecurity logs (`failregex = Modsecurity: .*denied.*<HOST>`). Finally, Fail2Ban is set up to automatically block malicious IP addresses by modifying the `/etc/fail2ban/jail.local` file, which defines the jail settings. This configuration allows Fail2Ban to automatically ban IPs identified as threats, providing real

![Hash passwords](imag/6_5.jpeg)

The image shows the configuration of a Fail2Ban filter, where a regular expression is set in `/etc/fail2ban/filter.d/modsec.conf` to detect denied requests in ModSecurity logs. The regex `failregex = Modsecurity: .*denied.*<HOST>` captures entries with "denied" followed by an IP address. This allows Fail2Ban to automatically block IP addresses attempting malicious activity, providing real-time protection.

![Hash passwords](imag/6_6.jpeg)

In the image, the Fail2Ban jail configuration is shown, specifically the section for ModSecurity (`/etc/fail2ban/jail.local`). It defines the settings for real-time monitoring by enabling Fail2Ban (`enabled = true`), setting the filter to `modsec`, and specifying the action to block malicious IP addresses using `iptables`. The log file to monitor is `/var/log/modsec_audit.log`, and the maximum number of retries before banning an IP is set to 5 (`maxretry = 5`). This configuration allows Fail2Ban to automatically block IP addresses attempting malicious actions, such as SQL injection or brute-force attacks, based on the ModSecurity logs, providing real-time protection against these threats.

![Hash passwords](imag/6_7.jpeg)

**Set Up Alerts for Suspicious Activity**

This image shows the configuration of alerts for suspicious activity in ModSecurity. The file `/etc/logwatch/conf/services/modsec.conf` is being edited to include keywords like `union`, `select`, `from`, `<script>`, and `document.cookie`, which are commonly associated with SQL injections or XSS attacks. By configuring such alerts, administrators are notified when potential security threats are detected, allowing for quick responses to mitigate risks before they escalate.


### Configure ModSecurity’s Detection Thresholds and Sensitivity

![Hash passwords](imag/6_8.jpeg)

In this image, the ModSecurity configuration file (`/etc/modsecurity/modsecurity.conf`) is being edited to enable critical security rules. The configuration includes:

- **SecRuleEngine On**: Activates ModSecurity and attaches it to every HTTP transaction.
- **SecRequestBodyAccess On**: Enables ModSecurity to inspect the request body.
- **SecResponseBodyAccess On**: Enables ModSecurity to inspect the response body.
- **SecRuleParanoiaLevel 3**: Sets the paranoia level to 3, which increases sensitivity to detect potentially harmful requests, while balancing the risk of false positives.

These settings ensure that ModSecurity is actively monitoring and protecting the web application from various security threats. Regularly reviewing and updating these rules is essential for maintaining an effective Web Application Firewall (WAF) that can adapt to new attack techniques and evolving vulnerabilities.

![Hash passwords](imag/6_9.jpeg)

The command `sudo nano /etc/modsecurity/modsecurity.conf` is executed to edit the ModSecurity configuration file. This step is part of the process of managing ModSecurity settings, which is essential for configuring the (WAF). By accessing this file, administrators can modify various rules, sensitivity levels, and other settings that control how ModSecurity detects and blocks malicious traffic, ensuring the system is properly secured.

![Hash passwords](imag/6_10.jpeg)

The image shows the configuration for log rotation in the file `/etc/logrotate.d/modsec`. Log rotation is a crucial practice for managing system resources and ensuring that log files do not consume excessive disk space. In this configuration, ModSecurity logs (`/var/log/modsec_audit.log`) are set to rotate daily, with up to 7 archived logs. The `compress` and `delaycompress` options ensure that older logs are compressed to save space. This process helps maintain manageable log sizes while retaining necessary data for analysis or troubleshooting purposes, without sacrificing the availability of historical logs.

### WAF Bypass Techniques and Their Effectiveness

In this section, we outline several advanced techniques used to bypass Web Application Firewalls (WAFs) and their effectiveness in evading detection. The following techniques were tested and evaluated:

1. **Double URL Encoding**: A technique where characters are URL-encoded twice to bypass WAF filters that decode URLs only once.
2. **HTTP Parameter Pollution (HPP)**: This technique involves injecting multiple parameters with the same name into a request, exploiting the way some WAFs process HTTP parameters.
3. **Case Sensitivity Bypass**: By altering the case of malicious payloads, attackers can exploit WAFs that perform case-sensitive evaluations of attack strings.
4. **Unicode Encoding Bypass**: Using Unicode encoding to bypass WAFs that may not decode Unicode characters properly, allowing malicious payloads to slip through.
5. **Comment Injection Bypass**: Injecting malicious code inside comments that some WAFs might ignore, allowing payloads to be executed when rendered in the browser.

These techniques were tested against the WAF setup to evaluate their effectiveness in bypassing WAF protections. The findings of these tests highlight the potential vulnerabilities in WAF configurations and provide insights into improving WAF detection mechanisms.

### Test your WAF against all DVWA

![Hash passwords](imag/6_6.5.jpeg)

This command is used to edit the log rotation configuration for ModSecurity logs. Log rotation ensures that log files are managed efficiently, preventing them from taking up excessive disk space. By using this configuration, old logs are archived, compressed, and kept for a defined period. This is important for maintaining system performance while retaining the necessary logs for troubleshooting and compliance.

In this context, `logrotate` will manage the ModSecurity audit logs, ensuring that logs are rotated regularly and older logs are archived for future reference or analysis.

![Hash passwords](imag/6_14.jpeg)

We can see an attempted **command injection** attack on the DVWA "Command Injection" module. The attacker tried to execute a reverse shell command by inserting a malicious payload into the input field. The system correctly responded with an error, indicating the IP address entered was invalid. This suggests that the WAF effectively blocked the command injection attempt, preventing a potential compromise. The WAF rule preventing such payloads was successful in filtering out this malicious request.

The WAF successfully blocked the command injection attempt, showing its efficiency at both medium and hard levels.

![Hash passwords](imag/6_15.jpeg)

The attacker attempted to upload a potentially dangerous file (`shell.php.png`) through the DVWA "File Upload" module. The file was rejected by the system because it was not a valid image format (JPEG or PNG), as shown by the error message. The WAF likely detected this as a potential file upload attack and blocked it, preventing the malicious file from being uploaded.

The WAF blocked the unauthorized file upload attempt, demonstrating its capacity to prevent attacks that exploit file upload vulnerabilities.


![Hash passwords](imag/6_16.jpeg)

This shows a series of SQL injection attempts, specifically targeting the DVWA "SQL Injection" module. These log entries contain evidence of an attacker trying to manipulate SQL queries through the input fields. The WAF logs indicate that the attempts did not succeed, as the requests were intercepted and blocked before they could exploit any vulnerabilities in the application.

The WAF effectively blocked SQL injection attempts, confirming that the system's rules for filtering SQL-related payloads are functioning as intended.

![Hash passwords](imag/6_17.jpeg)

The image shows ModSecurity's error logs and the Apache2 access logs. Here we can see errors related to ModSecurity’s inability to open the audit log file, likely due to incorrect file path configurations. These errors, such as "ModSecurity: Failed to open the audit log file," indicate issues with the ModSecurity configuration.

The error suggests that while the WAF might be blocking some attacks, there are configuration issues that prevent proper logging, which needs to be addressed to ensure better monitoring and tracking of attacks.


![Hash passwords](imag/6_18.jpeg)

The image shows the **ModSecurity status log** and Apache2 error logs, displaying the successful configuration of ModSecurity after resolving some previous issues. The logs indicate that ModSecurity is enabled and is running correctly, as evidenced by the `ModSecurity configured` messages. However, there are still warnings related to potential misconfigurations, such as the inability to fully resolve the server’s domain name and some missing configurations for the `SecStatusEngine` directive.

While ModSecurity is now configured and running, there are minor issues with name resolution and configuration that need to be addressed to optimize the WAF's performance. The WAF is currently functioning at a basic level but can be improved with further tuning.


In conclusion, although the WAF successfully blocked multiple attack attempts, including **command injection**, **file upload vulnerabilities**, and **SQL injections**, there are issues with ModSecurity's configuration and logging that need attention. These configuration errors, such as missing server name resolution and audit log issues, hinder the effectiveness of the WAF. Once these issues are fixed, the WAF will provide more reliable protection and offer detailed logging for further monitoring and analysis.

### Rule tuning documentation

![Hash passwords](imag/6_11.jpeg)

In the image, the command `sudo nano /etc/modsecurity/modsecurity.conf` is used to open the ModSecurity configuration file. This file contains the settings and rules that ModSecurity follows to filter and block malicious web requests. Using this command, system administrators can modify the configuration to suit their security needs, such as enabling or adjusting various rules to protect against specific vulnerabilities like SQL injection or cross-site scripting (XSS).

![Hash passwords](imag/6_12.jpeg)

![Hash passwords](imag/6_13.jpeg)

**Rule to Detect Password Hash Extraction Attempts**

In the images, we can see a **ModSecurity rule** designed to detect attempts to extract password hashes from a database. The rule examines various parts of the HTTP request, including the URI, headers, parameters, and request body, using regular expressions to detect malicious patterns.

- The rule uses the `@rx` operator to search for specific patterns in SQL queries, such as:
    - `select ... from ... users.password`: Detects SQL queries attempting to access the users' password column.
    - `union select`: Detects union-based SQL injection attempts to combine malicious queries.
    - `select ... from ... hash`: Looks for queries attempting to select password hashes.

This rule is configured with `id:10001` and `phase:2`, meaning it is applied during the second phase of request processing (request body analysis). If the rule matches any of these patterns, the request is denied, and an error message `Password hash extraction attempt detected` is logged.

This rule effectively blocks attempts to extract password hashes from the database, improving the security of the web application.


**ModSecurity Rule Explanation**

The configuration in the images shows a **ModSecurity rule** designed to prevent attackers from extracting password hashes from the database using SQL injection. The rule looks for specific patterns in the HTTP request, including attempts to query the `password` or `hash` fields from the `users` table using SQL injection techniques like `select`, `union select`, and other patterns designed to bypass security filters. When the rule detects these patterns, it blocks the request, logging the incident with a message indicating a password hash extraction attempt.

This rule provides a strong defense against SQL injection attacks targeting sensitive information, such as password hashes, and ensures that such attempts are promptly blocked by ModSecurity.



## Recommended WAF best practices

When it comes to protecting your digital/web assets, your Web Application Firewall (WAF) is one of your most critical security tools. It's important to understand how to deploy, monitor, and manage your WAF effectively to avoid confusion and ensure maximum protection. Below are some best practices to follow:

#### Deployment

Before deploying a WAF, carefully assess your needs and expectations. Consider these questions to set a solid foundation for a strong WAF deployment:
- What web applications and sensitive data do you want to protect?
- Does the WAF offer necessary features like API security, access control, and protection against OWASP Top 10 vulnerabilities?
- Which threats are you most concerned about? Focus on issues like Cross-site Scripting (XSS), SQL injection, and Denial-of-Service (DDoS) attacks.
- Does the WAF align with your organization's security policy, including features like IP whitelisting, rate limiting, and SSL support?

To ensure a successful WAF deployment:
- Document your applications, endpoints, and IP addresses.
- Determine your security risk tolerance.
- Define what traffic is allowed and restricted.

#### In-Line Deployment

WAFs can be deployed either in-line or out-of-line. **In-line deployment** is the preferred method, as it gives you more control over web traffic. This configuration allows for better security, enabling you to block malicious traffic and decrypt/log traffic as it passes through.

#### Monitoring and Management

Effective WAF monitoring is essential. Best practices for WAF management include:
- **Integrating with SIEM**: Ensure your WAF is integrated with your Security Information and Event Management (SIEM) system to provide broader visibility into network traffic.
- **Network Topology Awareness**: Understand your network architecture to improve monitoring and troubleshooting.
- **Developer Access**: Ensure your devops team has proper access to monitor and adjust the WAF as needed.

#### Regular Testing and Auditing

To maintain optimal security, regularly test your WAF against common threats like SQL injections and XSS. Stay updated on emerging threats and ensure your WAF is audit-ready for compliance standards (e.g., PCI DSS).

#### Rule Optimization

WAF rules should be regularly reviewed and tuned to ensure they do not produce false positives. It's essential to optimize the rules to balance security and usability, adapting them to your specific applications and deployments.

#### Integration with Other Security Tools

For comprehensive protection, integrate your WAF with other security tools such as Intrusion Prevention Systems (IPS) and DDoS protection services. A WAF integrated with a cloud
